In [7]:
import os
from keras import backend as K
from keras.engine.topology import Layer
from keras.models import Model
from keras.layers import Input, merge, BatchNormalization, Activation
from keras.layers.convolutional import Convolution2D, AveragePooling2D, MaxPooling2D, Deconvolution2D, Cropping2D

Using TensorFlow backend.
/home/amrit/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
#VCC16 function

In [8]:
class Denormalize(Layer):
    '''Custom layer to denormalize the final Convolution layer activations (tanh)
    Since tanh scales the output to the range (-1, 1), we add 1 to bring it to the
    range (0, 2). We then multiply it by 127.5 to scale the values to the range (0, 255)'''

    def __init__(self, **kwargs):
        super(Denormalize, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def call(self, x, mask=None):
        #Scales the tanh output activations from previous layer (-1, 1) to the range (0, 255)

        return (x + 1) * 127.5

    def get_output_shape_for(self, input_shape):
        return input_shape

In [9]:
def ResidualBlock(ip, id):
    init = ip

    #x = ReflectionPadding2D()(ip)
    x = Convolution2D(128, 3, 3, activation='linear', border_mode='valid', name='res_conv_' + str(id) + '_1')(ip)
    x = BatchNormalization(axis=1, name="res_batchnorm_" + str(id) + "_1")(x)
    x = Activation('relu', name="res_activation_" + str(id) + "_1")(x)

    #x = ReflectionPadding2D()(x)
    x = Convolution2D(128, 3, 3, activation='linear', border_mode='valid', name='res_conv_' + str(id) + '_2')(x)
    x = BatchNormalization(axis=1, name="res_batchnorm_" + str(id) + "_2")(x)

    m = merge([x, init], mode='sum', name="res_merge_" + str(id))
    #m = Activation('relu', name="res_activation_" + str(id))(m)

    return m

In [13]:
def ImageTransformationNet(deep_model=True):

    ip = Input(shape=(256, 256, 3), name="X_input")
    #c1 = ReflectionPadding2D((4, 4))(ip)

    c1 = Convolution2D(32, 9, 9, activation='linear', border_mode='valid', name='conv1')(ip)
    c1_b = BatchNormalization(axis=1, name="batchnorm1")(c1)
    c1_b = Activation('relu')(c1_b)

    c2 = Convolution2D(128, 3, 3, activation='linear', border_mode='same', subsample=(2, 2), name='conv2')(c1_b)
    c2_b = BatchNormalization(axis=1, name="batchnorm2")(c2)
    c2_b = Activation('relu')(c2_b)

    c3 = Convolution2D(128, 3, 2, activation='linear', border_mode='same', subsample=(2, 2), name='conv3')(c2_b)
    x = BatchNormalization(axis=1, name="batchnorm3")(c3)
    x = Activation('relu')(x)

    if deep_model:
        c4 = Convolution2D(128, 3, 3, activation='linear', border_mode='same', subsample=(2, 2), name='conv4')(x)
        x = BatchNormalization(axis=1, name="batchnorm_4")(c4)
        x = Activation('relu')(x)

    r1 = ResidualBlock(x, 1)
    r2 = ResidualBlock(r1, 2)
    r3 = ResidualBlock(r2, 3)
    r4 = ResidualBlock(r3, 4)
    x = ResidualBlock(r4, 5)

    if deep_model:
        d4 = Deconvolution2D(128, 3, 3, activation="linear", border_mode="same", subsample=(2, 2),
                             output_shape=(1, 64, 256 // 4, 256 // 4), name="deconv4")(x)

        x = BatchNormalization(axis=1, name="batchnorm_extra4")(d4)
        x = Activation('relu')(x)

    d3 = Deconvolution2D(128, 3, 3, activation="linear", border_mode="same", subsample=(2, 2),
                         output_shape=(1, 128, 256 // 2, 256 // 2),
                         name="deconv3")(x)

    d3 = BatchNormalization(axis=1, name="batchnorm4")(d3)
    d3 = Activation('relu')(d3)

    d2 = Deconvolution2D(128, 3, 3, activation="linear", border_mode="same", subsample=(2, 2),
                         output_shape=(1, 128, 256, 256), name="deconv2")(d3)

    d2 = BatchNormalization(axis=1, name="batchnorm5")(d2)
    d2 = Activation('relu')(d2)

    #d1 = ReflectionPadding2D((4, 4))(d2)
    d1 = Convolution2D(3, 9, 9, activation='tanh', border_mode='valid', name='fastnet_conv')(d1)

    # Scale output to range [0, 255] via custom Denormalize layer
    d1 = Denormalize(name='fastnet_output')(d1)

    model = Model(ip, d1)

In [14]:
model = ImageTransformationNet()

/home/amrit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (9, 9), activation="linear", name="conv1", padding="valid")`
  
/home/amrit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="linear", name="conv2", strides=(2, 2), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/home/amrit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 2), activation="linear", name="conv3", strides=(2, 2), padding="same")`
  
/home/amrit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="linear", name="conv4", strides=(2, 2), padding="same")`
/home/amrit/anaconda3/lib/python3.6/site-packages/ipykernel_l

ValueError: Only layers of same output shape can be merged using sum mode. Layer shapes: [(None, 27, 27, 128), (None, 31, 31, 128)]

In [5]:
def main():
    style_image = './dataset/style_images/'
    training_images = './dataset/training_images/'


In [6]:
if __name__ == '__main__':
    main()

img_13090.jpg
img_41317.jpg
img_20427.jpg
img_27328.jpg
img_31913.jpg
img_37550.jpg
img_24350.jpg
img_27302.jpg
img_14813.jpg
img_10289.jpg
img_45212.jpg
img_19006.jpg
img_34623.jpg
img_32110.jpg
img_22391.jpg
img_37186.jpg
img_15849.jpg
img_43092.jpg
img_12323.jpg
img_10694.jpg
img_1319.jpg
img_14637.jpg
img_44618.jpg
img_44571.jpg
img_25254.jpg
img_22515.jpg
img_12881.jpg
img_41610.jpg
img_26897.jpg
img_37607.jpg
img_47512.jpg
img_22266.jpg
img_40817.jpg
img_3282.jpg
img_3087.jpg
img_26951.jpg
img_2631.jpg
img_1168.jpg
img_46546.jpg
img_4341.jpg
img_4659.jpg
img_25703.jpg
img_46317.jpg
img_31760.jpg
img_35013.jpg
img_31718.jpg
img_15359.jpg
img_42541.jpg
img_23606.jpg
img_38042.jpg
img_405.jpg
img_17947.jpg
img_3270.jpg
img_44200.jpg
img_2759.jpg
img_33928.jpg
img_21215.jpg
img_20652.jpg
img_42739.jpg
img_289.jpg
img_46343.jpg
img_37084.jpg
img_10680.jpg
img_19515.jpg
img_39949.jpg
img_18986.jpg
img_16251.jpg
img_41893.jpg
img_31041.jpg
img_28061.jpg
img_2605.jpg
img_47035.jpg
img_37